# Insert data

In [1]:
from dotenv import load_dotenv

load_dotenv()

from IPython.display import Image, display
import ipyplot
import os
base_path = os.path.join(os.getcwd(), "mnist_features_128_dim_1000")
image_path = os.path.join(base_path, "production_data/unlabelled_prod_0")
features_file = os.path.join(base_path, "feature_results", "feature_results.json")
db_name = "mnist_features"
db_uri = os.getenv("MONGO_URI")
                    

## Connect to the database

In [2]:
from pymongo import MongoClient
client = MongoClient(db_uri)

db = client[db_name]

## Grab the features data

In [3]:
import json

with open(features_file) as f:
    data = json.load(f)

data[0]

{'image_path': 'production_data/unlabelled_prod_0/img_26134.jpg',
 'features': [0.0,
  10.9813814163,
  2.5166893005,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  12.1448707581,
  3.1732404232,
  0.0,
  4.5034627914,
  0.0,
  0.0,
  8.5474596024,
  0.0,
  0.0,
  7.2134747505,
  0.0,
  0.0,
  0.0,
  0.0,
  11.3414211273,
  0.0,
  0.0,
  0.0,
  0.0,
  5.8528933525,
  4.2120985985,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  12.7576494217,
  0.0,
  1.1928420067,
  0.0,
  2.3526725769,
  0.0,
  0.0,
  9.9314994812,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.3979257345,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  9.8350811005,
  1.9471485615,
  0.0,
  0.0,
  0.0,
  11.2871417999,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.2470169067,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  7.5622706413,
  6.3862671852,
  13.3446092606,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  7.2219810486,
  0.0,
  0.0,
  0.0,
  

## Insert into database

In [4]:
img_coll = db["images"]

for d in data:
    img_coll.insert_one(d)

In [5]:
img_coll.count_documents({})

1000

## Create the index

Create the database in the index
```
db.runCommand({"createIndexes": "images", 
    "indexes": [{ 
      "key": {"features":"vector"}, 
      "name": "vector_embeddings",
    "vectorOptions": {
      "dimensions": 128,
      "similarity": "euclidean",
      "lists": 1
    }
  }] 
})
```

## Check the data of a random image and visualize it

In [15]:
rec = img_coll.find_one({"image_path": "production_data/unlabelled_prod_0/img_5805.jpg"})
first_rec_features = rec["features"]
len(first_rec_features)

128

In [16]:
Image(filename=os.path.join(base_path, rec["image_path"])) 

## Find the closest X images

In [17]:
result = img_coll.aggregate([
    {
      "$search": {
        "vectorSearch": {
          "similarity": "euclidean",
          "path": "features",
          "vector": first_rec_features,
          "k": 30
        }
      }
  }
])
matches = [r for r in result]



In [18]:
imgs = [r["image_path"] for r in matches]
full_img_paths = [os.path.join(base_path, img) for img in imgs]
    
ipyplot.plot_images(full_img_paths, img_width=100)